# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb8f4152310>
├── 'a' --> tensor([[-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475]])
└── 'x' --> <FastTreeValue 0x7fb8f4152190>
    └── 'c' --> tensor([[-0.7724,  0.6396,  1.0891, -0.0686],
                        [ 0.6082,  0.2599,  0.4382, -0.6928],
                        [ 0.3257, -0.0069,  0.2315, -0.1226]])

In [4]:
t.a

tensor([[-0.4014, -0.9856, -1.0800],
        [-0.3798, -1.1935, -2.4475]])

In [5]:
%timeit t.a

65.3 ns ± 0.0447 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb8f4152310>
├── 'a' --> tensor([[-1.5062, -0.8381, -0.9479],
│                   [-1.0839,  0.7562, -1.7068]])
└── 'x' --> <FastTreeValue 0x7fb8f4152190>
    └── 'c' --> tensor([[-0.7724,  0.6396,  1.0891, -0.0686],
                        [ 0.6082,  0.2599,  0.4382, -0.6928],
                        [ 0.3257, -0.0069,  0.2315, -0.1226]])

In [7]:
%timeit t.a = new_value

74.3 ns ± 0.0294 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4014, -0.9856, -1.0800],
               [-0.3798, -1.1935, -2.4475]]),
    x: Batch(
           c: tensor([[-0.7724,  0.6396,  1.0891, -0.0686],
                      [ 0.6082,  0.2599,  0.4382, -0.6928],
                      [ 0.3257, -0.0069,  0.2315, -0.1226]]),
       ),
)

In [10]:
b.a

tensor([[-0.4014, -0.9856, -1.0800],
        [-0.3798, -1.1935, -2.4475]])

In [11]:
%timeit b.a

58.2 ns ± 0.105 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7499, -0.4032,  1.7621],
               [-1.5287, -0.1927,  0.2334]]),
    x: Batch(
           c: tensor([[-0.7724,  0.6396,  1.0891, -0.0686],
                      [ 0.6082,  0.2599,  0.4382, -0.6928],
                      [ 0.3257, -0.0069,  0.2315, -0.1226]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.114 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

904 ns ± 6.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 386 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb849dfbf40>
├── 'a' --> tensor([[[-0.4014, -0.9856, -1.0800],
│                    [-0.3798, -1.1935, -2.4475]],
│           
│                   [[-0.4014, -0.9856, -1.0800],
│                    [-0.3798, -1.1935, -2.4475]],
│           
│                   [[-0.4014, -0.9856, -1.0800],
│                    [-0.3798, -1.1935, -2.4475]],
│           
│                   [[-0.4014, -0.9856, -1.0800],
│                    [-0.3798, -1.1935, -2.4475]],
│           
│                   [[-0.4014, -0.9856, -1.0800],
│                    [-0.3798, -1.1935, -2.4475]],
│           
│                   [[-0.4014, -0.9856, -1.0800],
│                    [-0.3798, -1.1935, -2.4475]],
│           
│                   [[-0.4014, -0.9856, -1.0800],
│                    [-0.3798, -1.1935, -2.4475]],
│           
│                   [[-0.4014, -0.9856, -1.0800],
│                    [-0.3798, -1.1935, -2.4475]]])
└── 'x' --> <FastTreeValue 0x7fb850a1f250>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 49.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb850a1fa00>
├── 'a' --> tensor([[-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475],
│                   [-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475],
│                   [-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475],
│                   [-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475],
│                   [-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475],
│                   [-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475],
│                   [-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475],
│                   [-0.4014, -0.9856, -1.0800],
│                   [-0.3798, -1.1935, -2.4475]])
└── 'x' --> <FastTreeValue 0x7fb84a549070>
    └── 'c' --> tensor([[-0.7724,  0.6396,  1.0891, -0.0686],
                        [ 0.6082,  0.2599,  0.4382, -0.6928],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 166 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.6 µs ± 65.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.7724,  0.6396,  1.0891, -0.0686],
                       [ 0.6082,  0.2599,  0.4382, -0.6928],
                       [ 0.3257, -0.0069,  0.2315, -0.1226]],
              
                      [[-0.7724,  0.6396,  1.0891, -0.0686],
                       [ 0.6082,  0.2599,  0.4382, -0.6928],
                       [ 0.3257, -0.0069,  0.2315, -0.1226]],
              
                      [[-0.7724,  0.6396,  1.0891, -0.0686],
                       [ 0.6082,  0.2599,  0.4382, -0.6928],
                       [ 0.3257, -0.0069,  0.2315, -0.1226]],
              
                      [[-0.7724,  0.6396,  1.0891, -0.0686],
                       [ 0.6082,  0.2599,  0.4382, -0.6928],
                       [ 0.3257, -0.0069,  0.2315, -0.1226]],
              
                      [[-0.7724,  0.6396,  1.0891, -0.0686],
                       [ 0.6082,  0.2599,  0.4382, -0.6928],
                       [ 0.3257, -0.0069,  0.2315, -0.1226]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.7724,  0.6396,  1.0891, -0.0686],
                      [ 0.6082,  0.2599,  0.4382, -0.6928],
                      [ 0.3257, -0.0069,  0.2315, -0.1226],
                      [-0.7724,  0.6396,  1.0891, -0.0686],
                      [ 0.6082,  0.2599,  0.4382, -0.6928],
                      [ 0.3257, -0.0069,  0.2315, -0.1226],
                      [-0.7724,  0.6396,  1.0891, -0.0686],
                      [ 0.6082,  0.2599,  0.4382, -0.6928],
                      [ 0.3257, -0.0069,  0.2315, -0.1226],
                      [-0.7724,  0.6396,  1.0891, -0.0686],
                      [ 0.6082,  0.2599,  0.4382, -0.6928],
                      [ 0.3257, -0.0069,  0.2315, -0.1226],
                      [-0.7724,  0.6396,  1.0891, -0.0686],
                      [ 0.6082,  0.2599,  0.4382, -0.6928],
                      [ 0.3257, -0.0069,  0.2315, -0.1226],
                      [-0.7724,  0.6396,  1.0891, -0.0686],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 338 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

331 µs ± 795 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
